# Segmenting and Clustering Neighborhoods in Toronto

Created by: Sangwook Cheon

In this notebook, I will replicate the analysis done on neighborhoods in New York City. I will use boroughs that contain the text 'Toronto,' to only analyze this particular region.

## Import Libraries

In [2]:
import numpy as np  # library to handle data in a vectorized manner
import pandas as pd  # library for data analsysis
import json  # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Below I load a cleaned data table which I uploaded to my GitHub repository.

In [3]:
neighborhoods = pd.read_csv('https://raw.githubusercontent.com/SangwookCheon/non-e-stop-python-data-science/master/Machine-Learning/Example%20Projects/IBM%20Capstone%20Project/canada_cpdes_latlng.csv', header=0, index_col=0)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Each row has Latitude and Longitude values, which can be used to pinpoint exact locations on a map.

In [4]:
neighborhoods.iloc[0]

PostalCode             M3A
Borough         North York
Neighborhood     Parkwoods
Latitude           43.7533
Longitude         -79.3297
Name: 0, dtype: object

Now let's drop all the rows that do not belong to Toronto:

In [5]:
for i in range(0, len(neighborhoods)):
    if 'Toronto' not in neighborhoods.loc[i, 'Borough']:
        neighborhoods.drop(i, inplace=True)

In [6]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [8]:
neighborhoods.reset_index(drop=True, inplace=True)

In [9]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Now that we have neighborhoods in Toronto along with their locations, let's first visualize where they are on the map.

In [10]:
latitude = 43.6532
longitude = -79.3832

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

## Exploring Neighborhoods using Foursquare API

Now, using Foursquare API, let's go deeper into the neighborhoods in Toronto.
First, specify my access information for Foursquare API service:

In [36]:
CLIENT_ID = 'YBQKUPKIWQV3W4TW0HJ52NU5GNOOM5EOJROEOP0ZAE5HYU4C' # your Foursquare ID
CLIENT_SECRET = 'J2Z2XS3RHKLYPDZU0WYGDRAX1PMEFYP1LK2USEVPIWNXTVLA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Exploring one neighborhood

Start the analysis by analyzing the first neighborhood in the data table:

In [37]:
neighborhoods.loc[neighborhoods.index[0], 'Neighborhood']

'Harbourfront, Regent Park'

In [38]:
neighborhood_latitude = neighborhoods.loc[neighborhoods.index[0], 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[neighborhoods.index[0], 'Longitude'] # neighborhood longitude value

With the Latitude and Longitude value above, let's explore top 10 places in this neighborhood.

In [39]:
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YBQKUPKIWQV3W4TW0HJ52NU5GNOOM5EOJROEOP0ZAE5HYU4C&client_secret=J2Z2XS3RHKLYPDZU0WYGDRAX1PMEFYP1LK2USEVPIWNXTVLA&ll=43.6532,-79.3832&v=20180605&radius=500&limit=10'

In [40]:
results = requests.get(url).json()

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
4,CF Toronto Eaton Centre,Shopping Mall,43.653779,-79.380699
5,Old City Hall,Monument / Landmark,43.652009,-79.381744
6,M Square Coffee Co,Coffee Shop,43.651218,-79.383555
7,SEPHORA,Cosmetics Shop,43.653688,-79.380120
8,Noodle King,Asian Restaurant,43.651706,-79.383046
9,LUSH,Cosmetics Shop,43.653557,-79.380400


Above are the 10 trending venues.

In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


Now it's time to explore trending places for all the neighborhoods in Toronto. To do this, define a function that repeats the process shown above:

In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [73]:
print(toronto_venues.shape)
toronto_venues.head()

(341, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [74]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",10,10,10,10,10,10
Berczy Park,10,10,10,10,10,10
"Brockton, Exhibition Place, Parkdale Village",10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,10,10,10,10,10,10
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",10,10,10,10,10,10
"Cabbagetown, St. James Town",10,10,10,10,10,10
Central Bay Street,10,10,10,10,10,10
"Chinatown, Grange Park, Kensington Market",10,10,10,10,10,10
Christie,10,10,10,10,10,10


For most of the neighborhoods, we got 10 results. However, for some neighborhoods with less than 10 results, it is possible that there were not enough trending venues to return.

In [48]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 116 uniques categories.


One-hot encoding is used to use Clustering on this data table. 

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
toronto_onehot.shape

(338, 116)

In [52]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant,Dance Studio,Dessert Shop,Diner,Dog Run,Eastern European Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,General Entertainment,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Korean Restaurant,Lake,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Movie Theater,Museum,Music Venue,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Pub,Ramen Restaurant,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Skate Park,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.000000,0.0,0.0,0.00,0.100000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.1,0.00,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.1,0.0,0.0,0.1,0.00,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.1,0.0,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.1,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.00,0.000000,0.0,0.00,0.1,0.0,0.0,0.00,0.1,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.1,0.0,0.0,0.000000,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.00,0.000000,0.0,0.0,0.1,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.00,0.0,0.00,0.0,0.1,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [53]:
toronto_grouped.shape

(38, 116)

Let's print each neighborhood along with the top 5 most common venues

In [54]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                           venue  freq
0             Seafood Restaurant   0.1
1                      Speakeasy   0.1
2  Vegetarian / Vegan Restaurant   0.1
3               Greek Restaurant   0.1
4                          Plaza   0.1


----Berczy Park----
             venue  freq
0     Concert Hall   0.1
1  Thai Restaurant   0.1
2   Farmers Market   0.1
3       Steakhouse   0.1
4           Museum   0.1


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop   0.2
1  Breakfast Spot   0.1
2            Café   0.1
3             Gym   0.1
4       Pet Store   0.1


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0            Comic Shop   0.1
1            Restaurant   0.1
2           Pizza Place   0.1
3  Fast Food Restaurant   0.1
4         Burrito Place   0.1


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
     

It can be noticed that Toronto has a variety of restaurants with different styles, such as Greek, Eastern European, Sushi, Indian, American, etc. Cafe and Coffee Shop shows up often as one of the top 5 venues.

In [55]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Speakeasy,Greek Restaurant,Hotel,Steakhouse,Plaza,Concert Hall,Asian Restaurant,Food Court,Vegetarian / Vegan Restaurant,Seafood Restaurant
1,Berczy Park,Park,Liquor Store,Vegetarian / Vegan Restaurant,French Restaurant,Cocktail Bar,Thai Restaurant,Museum,Farmers Market,Steakhouse,Concert Hall
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Pet Store,Gym,Bar,Italian Restaurant,Café,Caribbean Restaurant,Furniture / Home Store,Breakfast Spot,Cosmetics Shop
3,Business Reply Mail Processing Centre 969 Eastern,Pizza Place,Skate Park,Farmers Market,Fast Food Restaurant,Auto Workshop,Restaurant,Burrito Place,Comic Shop,Garden Center,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Plane,Bar,Airport,Airport Food Court,Airport Gate,Airport Terminal,Coffee Shop,Harbor / Marina,Dog Run


Now, let's cluster the data table which was one-hot encoded.

In [56]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 0, 4, 0, 2, 0, 2, 2, 2], dtype=int32)

In [59]:
kmeans.labels_.max()

4

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Breakfast Spot,Gym / Fitness Center,Coffee Shop,Pub,Park,Bakery,Restaurant,Spa,Cosmetics Shop,Concert Hall
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Diner,Tea Room,Pizza Place,Café,Comic Shop,Plaza,Burrito Place,Ramen Restaurant,Clothing Store,Theater
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Italian Restaurant,BBQ Joint,Gastropub,Japanese Restaurant,Food Truck,Creperie,Middle Eastern Restaurant,Restaurant,Dessert Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Pub,Health Food Store,Wine Bar,Eastern European Restaurant,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Park,Liquor Store,Vegetarian / Vegan Restaurant,French Restaurant,Cocktail Bar,Thai Restaurant,Museum,Farmers Market,Steakhouse,Concert Hall


In [58]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
toronto_merged.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Breakfast Spot,Gym / Fitness Center,Coffee Shop,Pub,Park,Bakery,Restaurant,Spa,Cosmetics Shop,Concert Hall
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Diner,Tea Room,Pizza Place,Café,Comic Shop,Plaza,Burrito Place,Ramen Restaurant,Clothing Store,Theater
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Italian Restaurant,BBQ Joint,Gastropub,Japanese Restaurant,Food Truck,Creperie,Middle Eastern Restaurant,Restaurant,Dessert Shop
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Trail,Pub,Health Food Store,Wine Bar,Eastern European Restaurant,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,Park,Liquor Store,Vegetarian / Vegan Restaurant,French Restaurant,Cocktail Bar,Thai Restaurant,Museum,Farmers Market,Steakhouse,Concert Hall
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Park,Tea Room,Sushi Restaurant,Modern European Restaurant,Pizza Place,Gastropub,Italian Restaurant,College Gym,Comfort Food Restaurant
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Café,Grocery Store,Diner,Coffee Shop,Italian Restaurant,Baby Store,Restaurant,Cuban Restaurant,Dessert Shop,Dance Studio
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568,4,Speakeasy,Greek Restaurant,Hotel,Steakhouse,Plaza,Concert Hall,Asian Restaurant,Food Court,Vegetarian / Vegan Restaurant,Seafood Restaurant
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259,2,Bakery,Pharmacy,Café,Bar,Supermarket,Gym / Fitness Center,Brewery,Middle Eastern Restaurant,Music Venue,Cosmetics Shop
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752,2,Park,Performing Arts Venue,Bubble Tea Shop,Sporting Goods Shop,Bakery,Salad Place,Café,Lake,Supermarket,Creperie


Notice any patterns? To better understand what each cluster mean, let's analyze cluster by cluster.

## Cluster 1

In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Italian Restaurant,BBQ Joint,Gastropub,Japanese Restaurant,Food Truck,Creperie,Middle Eastern Restaurant,Restaurant,Dessert Shop
5,Downtown Toronto,0,Coffee Shop,Park,Tea Room,Sushi Restaurant,Modern European Restaurant,Pizza Place,Gastropub,Italian Restaurant,College Gym,Comfort Food Restaurant
13,West Toronto,0,Coffee Shop,Pet Store,Gym,Bar,Italian Restaurant,Café,Caribbean Restaurant,Furniture / Home Store,Breakfast Spot,Cosmetics Shop
22,Central Toronto,0,Coffee Shop,Yoga Studio,Spa,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant,Salon / Barbershop,Chinese Restaurant,Asian Restaurant
24,West Toronto,0,Dog Run,Eastern European Restaurant,Gift Shop,Coffee Shop,Bar,Italian Restaurant,Movie Theater,Restaurant,Dessert Shop,Cuban Restaurant
30,Central Toronto,0,Coffee Shop,Fried Chicken Joint,Sushi Restaurant,Sports Bar,Restaurant,Liquor Store,Supermarket,Pub,American Restaurant,Cosmetics Shop
31,Downtown Toronto,0,Airport Lounge,Plane,Bar,Airport,Airport Food Court,Airport Gate,Airport Terminal,Coffee Shop,Harbor / Marina,Dog Run


Coffee Shop seems to be the dominant factor determining this cluster.

## Cluster 2

In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,1,Park,Bus Line,Swim School,Wine Bar,Dog Run,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop
28,Central Toronto,1,Park,Playground,Tennis Court,Summer Camp,Wine Bar,Diner,Coffee Shop,College Gym,Comfort Food Restaurant,Comic Shop
32,Downtown Toronto,1,Park,Trail,Playground,Building,Wine Bar,Dog Run,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall


Park shows up as the 1st common venue. On the map, we can see that these neighborhoods are indeed close to areas that are colored green.

## Cluster 3

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Breakfast Spot,Gym / Fitness Center,Coffee Shop,Pub,Park,Bakery,Restaurant,Spa,Cosmetics Shop,Concert Hall
1,Downtown Toronto,2,Diner,Tea Room,Pizza Place,Café,Comic Shop,Plaza,Burrito Place,Ramen Restaurant,Clothing Store,Theater
6,Downtown Toronto,2,Café,Grocery Store,Diner,Coffee Shop,Italian Restaurant,Baby Store,Restaurant,Cuban Restaurant,Dessert Shop,Dance Studio
8,West Toronto,2,Bakery,Pharmacy,Café,Bar,Supermarket,Gym / Fitness Center,Brewery,Middle Eastern Restaurant,Music Venue,Cosmetics Shop
9,Downtown Toronto,2,Park,Performing Arts Venue,Bubble Tea Shop,Sporting Goods Shop,Bakery,Salad Place,Café,Lake,Supermarket,Creperie
12,Downtown Toronto,2,Café,Japanese Restaurant,Coffee Shop,Restaurant,Hotel,Pub,Gym,Gym / Fitness Center,Gastropub,Cosmetics Shop
15,Downtown Toronto,2,Café,Gym / Fitness Center,Gastropub,Coffee Shop,Museum,American Restaurant,Gym,Restaurant,Pub,Cosmetics Shop
16,East Toronto,2,Bookstore,Ice Cream Shop,Comfort Food Restaurant,Chinese Restaurant,Cheese Shop,Café,Sandwich Place,Fish Market,Coffee Shop,Flea Market
21,West Toronto,2,Speakeasy,Arts & Crafts Store,Music Venue,Café,Mexican Restaurant,Italian Restaurant,Bar,Flea Market,Park,Gastropub
23,Central Toronto,2,Café,Pizza Place,American Restaurant,Burger Joint,Coffee Shop,Indian Restaurant,BBQ Joint,Park,Vegetarian / Vegan Restaurant,Food & Drink Shop


Cafe shows up quite often, and it seems that majority of common venues are related to food. There are a lot of these in the center of Toronto. Is the center the most popular place to open a shop related to food? If yes, why?

## Cluster 4

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,3,Garden,Wine Bar,Eastern European Restaurant,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cuban Restaurant


## Cluster 5

In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Trail,Pub,Health Food Store,Wine Bar,Eastern European Restaurant,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop
4,Downtown Toronto,4,Park,Liquor Store,Vegetarian / Vegan Restaurant,French Restaurant,Cocktail Bar,Thai Restaurant,Museum,Farmers Market,Steakhouse,Concert Hall
7,Downtown Toronto,4,Speakeasy,Greek Restaurant,Hotel,Steakhouse,Plaza,Concert Hall,Asian Restaurant,Food Court,Vegetarian / Vegan Restaurant,Seafood Restaurant
10,West Toronto,4,Wine Bar,Asian Restaurant,Greek Restaurant,Ice Cream Shop,Korean Restaurant,Cuban Restaurant,Cocktail Bar,Brewery,Bar,Pizza Place
11,East Toronto,4,Greek Restaurant,Ice Cream Shop,Yoga Studio,Pub,Cosmetics Shop,Italian Restaurant,Brewery,Dog Run,Comfort Food Restaurant,Comic Shop
14,East Toronto,4,Italian Restaurant,Fish & Chips Shop,Park,Ice Cream Shop,Burger Joint,Liquor Store,Pub,Gym,Brewery,Sushi Restaurant
19,Central Toronto,4,Clothing Store,Breakfast Spot,Park,Gym,Food & Drink Shop,Hotel,Sandwich Place,Comfort Food Restaurant,Comic Shop,Concert Hall
20,Central Toronto,4,Trail,Sushi Restaurant,Mexican Restaurant,Jewelry Store,Wine Bar,Dog Run,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall
25,Central Toronto,4,Dessert Shop,Park,Café,Italian Restaurant,Indian Restaurant,Sushi Restaurant,Seafood Restaurant,Pizza Place,Coffee Shop,Comic Shop
27,West Toronto,4,Sushi Restaurant,Food,Pub,Coffee Shop,Fish & Chips Shop,Burrito Place,Tea Room,Bookstore,Café,Cosmetics Shop


There are more interesting questions to consider from this analysis. Thank you for reading this notebook!